In [22]:
import os
import re
import numpy as np
import pandas as pd
import json
from pprint import pprint
from tqdm import tqdm_notebook as tqdm
from multiprocessing import Pool, cpu_count

### Determining food names and tables

In [29]:
root = "../raw/www.valori-alimenti.com"
foods = [food for food in os.listdir(root) if food[0] != "."]

### Merging csv files

In [30]:
def parse_food(food):
    root = "../raw/www.valori-alimenti.com"
    file_path = "{path}/{name}/0.csv".format(path=root, name=food)
    df = pd.read_csv(file_path, index_col=0)
    if df.empty:
        return None
    try:
        for index, value in df.iterrows():
            if pd.isna(value).any():
                df = df.drop(index)
        for index, (unit, value) in df.iterrows():
            if isinstance(value, str) and "tr" in value:
                value = 0.05
            if unit not in ["mg", "mcg", "g", "kcal"]:
                df = df.iloc[[0] + list(range(2, len(df.columns)))]
                continue
            if unit in ["mg", "mcg"]:
                if unit == "mg":
                    df.loc[index] = float(value) * 1e-3
                elif unit == "mcg":
                    df.loc[index] = float(value) * 1e-6
                unit = "g"
            df = df.rename(
                index={
                    index:
                    "{index} | {unit}".format(index=index, unit=unit).lower()
                    .strip().replace("  ", " ")
                })
    except Exception as e:
        pprint(food)
        pprint(df)
        raise e
    df = df.drop(columns=["1"])
    df = df.transpose()
    with open("{path}/{name}/metadata.json".format(path=root, name=food),
              "r") as f:
        df["category"] = json.load(f)["category"]
    df.index = [food]
    df.index.name = "name"
    del df.columns.name
    return df

In [ ]:
from IPython.display import display
with Pool(cpu_count()) as p:
    valori_alimenti = pd.concat(list(tqdm(p.imap(parse_food, foods), total=len(foods))))
valori_alimenti.index.name = "name"
valori_alimenti.to_csv("../csv/valori_nutrizionali.csv")

In [ ]:
valori_alimenti